# Investment Report Generator

[![uv](https://img.shields.io/endpoint?url=https://raw.githubusercontent.com/astral-sh/uv/main/assets/badge/v0.json)](https://github.com/astral-sh/uv)
[![Ruff](https://img.shields.io/endpoint?url=https://raw.githubusercontent.com/astral-sh/ruff/main/assets/badge/v2.json)](https://github.com/astral-sh/ruff)
![Docker Image Version (tag)](https://img.shields.io/docker/v/mertssmnoglu/upsonic-investment-report-generator/latest)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mertssmnoglu/upsonic-investment-report-generator/blob/main/notebooks/investment_report_generator.ipynb)

Investment Report Generator example for [Upsonic](https://github.com/upsonic/upsonic) framework.

## Introduction

This advanced example shows how to build a sophisticated investment analysis system that combines market research, financial analysis, and portfolio management. The workflow uses a three-stage approach:

- Comprehensive stock analysis and market research
- Investment potential evaluation and ranking
- Strategic portfolio allocation recommendations

Key capabilities:

- Real-time market data analysis
- Professional financial research
- Investment risk assessment
- Portfolio allocation strategy
- Detailed investment rationale

Example companies to analyze:

- “AAPL, MSFT, GOOGL” (Tech Giants)
- “NVDA, AMD, INTC” (Semiconductor Leaders)
- “TSLA, F, GM” (Automotive Innovation)
- “JPM, BAC, GS” (Banking Sector)
- “AMZN, WMT, TGT” (Retail Competition)
- “PFE, JNJ, MRNA” (Healthcare Focus)
- “XOM, CVX, BP” (Energy Sector)

In [ ]:
# Install required packages
# notebooks/requirements.txt file is dedicated to run this notebook,
# and it contains the necessary dependencies to run this notebook such as
# `ipykernel` and `ipython`.
%pip install -r requirements.txt

In [ ]:
# Required Packages
from upsonic import Agent, Task
import yfinance as yf
from typing import Literal
from datetime import datetime
from pathlib import Path
from os import getenv

In [ ]:
# Tools

## yFinance
def yFinance(query: str):
    """
    Search Yfinance for the given query and return text results.

    Args:
        query: The search symbol to query like MSFT

    Returns:
        Dictionaries containing search results
    """

    data = yf.Ticker(query)

    return data.info


## Report Generation
class Reports:
    """
    Reports directory file read write operations.
    """

    @staticmethod
    def create_markdown_report(
        agent_role: Literal["stock-analyst", "research-analyst", "investment-lead"],
        content: str,
    ):
        """
        Creates a .md document in the correct directory with the date time string

        Args:
            agent_role:
                Must be one of stock-analyst, research-analyst, investment-lead.
                Only pass your own exact role value
            content: String content to write to file
        Returns:
            A message indicating success or failure
        """

        dir = Path("reports")

        # Check if it exists and is a directory
        if dir.exists() and dir.is_dir():
            for _ in dir.iterdir():
                pass
        else:
            print("Directory does not exist or is not a directory.")

        prefix = ""
        is_demo = getenv("DEMO")
        if is_demo is not None and is_demo.lower() == "true":
            prefix = "demo_"

        if agent_role == "stock-analyst":
            pathname = f"{prefix}stock_analyst_report_{dateTimeString()}.md"
        elif agent_role == "research-analyst":
            pathname = f"{prefix}research_analyst_report_{dateTimeString()}.md"
        elif agent_role == "investment-lead":
            pathname = f"{prefix}investment_leader_report_{dateTimeString()}.md"
        else:
            pathname = f"{prefix}report{dateTimeString()}.md"

        dir = dir.joinpath(pathname)

        return dir.write_text(content, encoding="utf-8")


def dateTimeString() -> str:
    return datetime.now().strftime("%Y%m%d%H%M%S")


In [ ]:
# User Input
print("Enter the tickers you want to analyze.")
print("Separate each ticker with a comma (e.g., XOM,CVX,BP):")
ticker = input("Ticker: ")

In [ ]:
# Stock Analyst Agent
stock_analyst = Agent(
    agent_id_="stock-analyst",
    name="Stock Analyst Agent",
    model="openai/gpt-4o",
    system_prompt="""
You are a stock analyst with expertise in financial modeling, valuation, and industry trends. Your task is to analyze publicly traded companies using the most recent news articles, earnings reports, SEC filings, and analyst commentary.

For each company:
- Extract and summarize key financial metrics (e.g., revenue, EBITDA, EPS, debt ratios, cash flow).
- Highlight recent performance trends, catalysts, or challenges from credible sources.
- Assess short-term and long-term risks, including macroeconomic, sectoral, and company-specific risks.
- Note any major events (e.g., M&A, litigation, regulatory issues, executive changes).

Keep the analysis objective, data-driven, and well-cited. Focus on facts and financial impacts.

**CRITICAL AND ABSOLUTELY MANDATORY RULE:**
Your sole and exclusive agent role is `stock-analyst`.

If you use a tool or function that has an `agent_role` parameter, you **MUST ONLY** and precisely pass your exact role value: `stock-analyst`.

**UNDER NO CIRCUMSTANCES SHOULD YOU CALL, DELEGATE TASKS TO, OR UTILIZE ANY OTHER ROLES** (e.g., `research-analyst` or `investment-lead`). This rule is to be followed without exception.
""",
)


stock_analyst_response = stock_analyst.do(
    Task(
        f"Generate comprehensive stock market analysis report in markdown format for {ticker}",
        tools=[yFinance, Reports.create_markdown_report],
    )
)

In [ ]:
# Research Analyst Agent
research_analyst = Agent(
    agent_id_="research-analyst",
    name="Research Analyst Agent",
    model="openai/gpt-4o",
    system_prompt="""
You are a research analyst who interprets data from stock analysts to create a deeper investment thesis. You are skilled in sector comparison, financial risk modeling, and behavioral finance.

Your responsibilities:
- Synthesize the company-specific analysis into a broader sector and macroeconomic context.
- Evaluate valuation (e.g., P/E, EV/EBITDA, DCF) compared to peers and historical norms.
- Quantify risk using qualitative and quantitative indicators.
- Identify investment themes, pricing inefficiencies, and sentiment factors.
- Flag red/yellow/green investment indicators based on your synthesis.

Deliver your output as a structured, decision-oriented brief. Be clear about your conviction level.

**CRITICAL AND ABSOLUTELY MANDATORY RULE:**
Your sole and exclusive agent role is `research-analyst`.

If you use a tool or function that has an `agent_role` parameter, you **MUST ONLY** and precisely pass your exact role value: `research-analyst`.

**UNDER NO CIRCUMSTANCES SHOULD YOU CALL, DELEGATE TASKS TO, OR UTILIZE ANY OTHER ROLES** (e.g., `stock-analyst` or `investment-lead`). This rule is to be followed without exception.
""",
)

research_analyst_response = research_analyst.do(
    Task(
        f"""As a researcher, rank the companies and generate detailed 'investment analysis and ranking' report in markdown format" \
        The stock market analyst's output is
        {stock_analyst_response}""",
        tools=[yFinance, Reports.create_markdown_report],
    )
)

In [ ]:
# Investment Lead Agent
investment_lead = Agent(
    agent_id_="investment-lead",
    name="Investment Leader Agent",
    model="openai/gpt-4o",
    system_prompt="""
You are an investment lead at a buy-side firm. Your role is to develop a strategic investment thesis using input from research and stock analysts. You must integrate risk, return, and portfolio fit to craft a final recommendation.

Responsibilities:
- Design a portfolio strategy (e.g., long/short, sector rotation, hedging tactics) using the provided analysis.
- Justify the inclusion, weighting, or exclusion of the stock in a model portfolio.
- Define the investment rationale, including time horizon, expected return, downside risk, and exit strategy.
- Prepare a final decision memo or investment committee summary with a recommendation (Buy, Hold, Sell, or Watch).
- Suggest monitoring criteria and future review triggers (e.g., earnings, macro shifts).

Prioritize clarity, defensibility, and actionable insights. Your output should guide actual capital allocation.

**CRITICAL AND ABSOLUTELY MANDATORY RULE:**
Your sole and exclusive agent role is `investment-lead`.

If you use a tool or function that has an `agent_role` parameter, you **MUST ONLY** and precisely pass your exact role value: `investment-lead`.

**UNDER NO CIRCUMSTANCES SHOULD YOU CALL, DELEGATE TASKS TO, OR UTILIZE ANY OTHER ROLES** (e.g., `stock-analyst` or `research-analyst`). This rule is to be followed without exception.
""",
)

investment_lead_response = investment_lead.do(
    Task(
        f"""As an Invesment Leader, generate complete investment report in markdown format
        
        The research analyst's output is
        {research_analyst_response}""",
        tools=[Reports.create_markdown_report],
    )
)